In [14486]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from dash import Dash,dcc, html, callback,Input,Output,State,callback_context,no_update
import dash_bootstrap_components as dbc
import dash_ag_grid as dag
import plotly.graph_objects as go

#### In this small project i want to make analyzing using oop

**Let's start!**

In [14487]:
app = Dash(__name__,external_stylesheets=[dbc.themes.MORPH])
app.config.suppress_callback_exceptions = True

In [14488]:
df = pd.read_csv("Sample - Superstore.csv",encoding='cp1252') # <- Just reading file

In [14489]:
dfinfo = pd.read_csv('Info.csv')

In [14490]:
df.isna().sum() # We haven't any null here and we can continue work with this project

Row ID           0
Order ID         0
Order Date       0
Ship Date        0
Ship Mode        0
Customer ID      0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Postal Code      0
Region           0
Product ID       0
Category         0
Sub-Category     0
Product Name     0
Sales            0
Quantity         0
Discount         0
Profit           0
dtype: int64

In [14491]:
df.head(1)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136


In [14492]:
fontcolor1 = '#f5cbcb'
fontcolor2 = '#00008B'
cardcolor = '#4069E1'
second_layercol = '#C0C0C0'

In [14493]:
def short_name(str_):
    str__ = str_.split()
    if len(str__) <= 1:
        return str_
    else:
        str__ = f'{str__[0]} {str__[1]}'
    return str(str__)

In [14494]:
df['short name'] = df['Product Name'].apply(short_name)

In [14495]:
df_cat = df.groupby('Category')['Sales'].sum().reset_index()

In [14496]:
df_cat

,Category,Sales
0,Furniture,741999.7953
1,Office Supplies,719047.0320
2,Technology,836154.0330


## Page 0

In [14497]:
df_data = df.copy()

In [14498]:
df_data['Order Date'] = pd.to_datetime(df_data['Order Date'])
df_data = df_data[['Order Date','Sales','Profit','State']]

In [14499]:
df_data[df_data['Order Date'].between('2014-01-03','2014-01-05')]

,Order Date,Sales,Profit,State
739,2014-01-04,11.784,4.2717,Illinois
740,2014-01-04,272.736,-64.7748,Illinois
741,2014-01-04,3.540,-5.4870,Illinois
1759,2014-01-05,19.536,4.8840,Pennsylvania
7980,2014-01-03,16.448,5.5512,Texas


In [14500]:
df_data = df_data.groupby(['Order Date','State'])[['Sales','Profit']].sum().reset_index()

In [14501]:
df_data['date_num'] = (df_data['Order Date']-pd.to_datetime('2014-01-03')).dt.days

In [14502]:
df_data = df_data.sort_values(by='Order Date')

In [14503]:
df_data.head()

,Order Date,State,Sales,Profit,date_num
0,2014-01-03,Texas,16.448,5.5512,0
1,2014-01-04,Illinois,288.060,-65.9901,1
2,2014-01-05,Pennsylvania,19.536,4.8840,2
3,2014-01-06,California,19.440,9.3312,3
4,2014-01-06,Georgia,12.780,5.2398,3


In [14504]:
marks = {i : date.strftime(r'%Y-%m-%d') for i,date in zip(df_data['date_num'],df_data['Order Date'])}

In [14505]:
class MyCard:
    def __init__(self,card_id,text_id,title):
        self.card_id = card_id
        self.text_id = text_id
        self.title = title
    def create_card(self):
        return dbc.Card(
            dbc.CardBody([
                html.H4(self.title,className='card-title text-center',style={'color':fontcolor1}),
                html.H4(id = self.text_id,className='card-text text-center mt-3',style={'color':fontcolor1}),
                html.I(className="bi bi-info-circle-fill me-2"),
            ]),
            id = self.card_id,
            color = fontcolor2,
            className='border border-white border border-2 rounded-1',
            style={'margin-left': '-10px', 
                   'margin-right': '-10px',
                   'height':'130px',
                   },
            inverse=True,
        )

In [14506]:
card1 = MyCard('card1','text-card1','Total Sales').create_card()
card2 = MyCard('card2','text-card2','Total Profit').create_card()
card3 = MyCard('card3','text-card3','Average day profit').create_card()
card4 = MyCard('card4','text-card4','Average day Sales').create_card()
card5 = MyCard('card5','text-card5','Average Order Value').create_card()
card6 = MyCard('card6','text-card6','Top Customer').create_card()
card7 = MyCard('card7','text-card7','Repeat Customer Rate').create_card()
card8 = MyCard('card8','text-card8','Average Shipping Time').create_card()


In [14507]:
marks = {}

# Добавление меток для каждого года
for year in range(df_data['Order Date'].dt.year.min(), df_data['Order Date'].dt.year.max() + 1):
    # Первый день года
    date_start = pd.to_datetime(f'{year}-01-01')
    date_num_start = (date_start - df_data['Order Date'].min()).days
    marks[date_num_start] = str(year)
    
    # Полугодие (второе полугодие)
    date_mid = pd.to_datetime(f'{year}-07-01')
    date_num_mid = (date_mid - df_data['Order Date'].min()).days
    marks[date_num_mid] = f'{year}-H2'

In [14508]:
range_slider_data1 = dcc.RangeSlider(
    id='range-slider',
    min=df_data['date_num'].min(),
    max=df_data['date_num'].max(),
    step=1,
    value=[df_data['date_num'].min(), df_data['date_num'].max()],
    marks=marks,
    updatemode='mouseup',  # Обновление значения только после отпускания мыши
    included=True,  # Показать промежуток между ползунками
    className="p-3",  # Отступы вокруг слайдера
    vertical=False  # Горизонтальный слайдер
)

# Применение дополнительных CSS-стилей через style
range_slider_data1.style = {
        'width': '90%',  # Ширина слайдера
        'margin': '20px auto',  # Центрируем слайдер и добавляем отступы сверху и снизу
        'padding': '10px',  # Внутренние отступы
        'border': '2px solid #007bff',  # Толщина и цвет границы слайдера
        'border-radius': '12px',  # Скругленные углы границы
        'box-shadow': '0 4px 8px rgba(0, 0, 0, 0.1)',  # Тень вокруг слайдера
        'background-color': '#e9ecef',  # Фоновый цвет слайдера
        'font-size': '16px',  # Размер текста для меток
        'font-family': 'Arial, sans-serif',  # Шрифт для меток
        'color': '#333333',  # Цвет текста меток
        'outline': 'none',  # Убираем обводку при фокусе
        'height': '45px'  # Высота слайдера
}

In [14509]:

page_0 = dbc.Container(
    [
    dbc.Row([
    html.H1('Category sales', 
            className='text-center mt-4  ',
            style={'color': fontcolor2},
            ),
    html.Hr(style={
    'border': 'none',
    'border-top': '8px solid',
    'width': '100%',
    'margin-top': '20px',
    'margin-bottom': '20px' 
})
    ]),
dbc.Row([
    dbc.Col(dcc.RadioItems(
        ['Sales','Profit'],id='sales_or_profit',
        value='Sales',
        inline=True,
        className='text-left mt-4 mb-4 mx-0',
        style={'color': fontcolor2, 'font-weight': 'bold','font-size': '18px'},
        inputStyle={
                    "margin-right":"12px",
                    "margin-left": "20px", 
                    "transform": "scale(1.5)",
                    "accent-color": fontcolor2}
        )),
    dbc.Col(),
    dbc.Col(dcc.RadioItems(
        ['In %','Average Profit'],id='coef_or_profit',value='In %',
        inline=True,
                className='text-left mt-4 mb-4 mx-0',
        style={'color': fontcolor2, 'font-weight': 'bold','font-size': '18px'},
        inputStyle={
                    "margin-right":"12px",
                    "margin-left": "20px", 
                    "transform": "scale(1.5)",
                    "accent-color": fontcolor2},
        ),        
        )
    ]),
dbc.Row([
    dbc.Col(dcc.Graph(id='categories',style={'height': '65vh'}),
                  width={"size": 4}),
    dbc.Col(dcc.Graph(id='sub-categories',style={'height': '65vh'}),width={"size": 4,}),
    dbc.Col([
        dcc.Graph(id='one-sales-profit',style={'height': '60vh'}),        
        ],width=4),
    ]),
dbc.Row([
    html.Hr(style={
    'border': 'none',
    'border-top': '8px solid',
    'width': '100%',
    'margin-top': '20px',
    'margin-bottom': '20px' 
})
]),
dbc.Row([
    dbc.Col(card1),
    dbc.Col(card2),
    dbc.Col(card4),
    dbc.Col(card3),
]),
dbc.Row([
    dbc.Col(card5),
    dbc.Col(card6),
    dbc.Col(card7),
    dbc.Col(card8),
]),
dbc.Row([
    dbc.Col(range_slider_data1),
]),
dbc.Row([
    dbc.Col(html.Div(id='slider-output', className='mt-4')),
        ]),
   ],fluid=True
   )

## Page 1

In [14510]:
superstore_cities = pd.read_csv("Superstore cities.csv")

In [14511]:
range_slider_data2 = dcc.RangeSlider(
    id='range-slider2',
    min=df_data['date_num'].min(),
    max=df_data['date_num'].max(),
    step=1,
    value=[df_data['date_num'].min(), df_data['date_num'].max()],
    marks=marks,
    updatemode='mouseup',  # Обновление значения только после отпускания мыши
    included=True,  # Показать промежуток между ползунками
    className="p-3",  # Отступы вокруг слайдера
    vertical=False  # Горизонтальный слайдер
)

# Применение дополнительных CSS-стилей через style
range_slider_data2.style = {
    'margin': '20px 0',  # Отступ сверху и снизу
    'padding': '10px',  # Внутренние отступы
    'border': '1px solid #007bff',  # Граница слайдера
    'border-radius': '8px',  # Скругленные углы
    'box-shadow': '2px 2px 10px rgba(0, 123, 255, 0.2)',  # Тень
    'background-color': '#f8f9fa',  # Фоновый цвет слайдера
}

In [14512]:
import json
with open('state_codes.json', 'r') as f:
    state_codes = json.load(f)

In [14513]:
df_states_w_city = df.drop(['Order Date','short name'],axis=1).groupby(['State','City']).sum()[['Sales','Profit']].reset_index()

In [14514]:
df_states_w_city['State code'] = df_states_w_city['State'].map(state_codes)

In [14515]:
df_states_no_city = df_states_w_city.drop('City',axis=1).groupby(['State','State code']).sum().reset_index()

In [14516]:
df_states_no_city.head()

,State,State code,Sales,Profit
0,Alabama,AL,19510.6400,5786.8253
1,Arizona,AZ,35282.0010,-3427.9246
2,Arkansas,AR,11678.1300,4008.6871
3,California,CA,457687.6315,76381.3871
4,Colorado,CO,32108.1180,-6527.8579


In [14517]:
df_states_no_city['Profit'] = df_states_no_city['Profit'].round().apply(lambda x: f'{x} $')

In [14518]:
df_states_no_city['Sales'] = df_states_no_city['Sales'].round()

In [14519]:
page_1 = dbc.Container([
    dbc.Row([
    html.H3('Sales map', className='text-left'),
    ]),
    dbc.Row([
    dbc.Col(dcc.Graph(figure={},id='map',
            style={'padding': '0px', 'margin': '0px','height': '70vh'},
             className='container-fluid')
    ),
    dbc.Row([
        dbc.Col(range_slider_data2)
    ])
    ]),
    dbc.Row([
        dbc.Col(
            dcc.Graph(figure = {},id='map_state',
            style={'padding': '0px', 'margin': '0px','height': '45vh'},
            className='container-fluid'),
        ),
        dbc.Col(
        dcc.Graph(figure = {},id='sub-category-in-state',
                 className='container-fluid',
                 style={'padding': '0px', 'margin': '0px','height': '45vh'},),    
        ),
        ]), 
],fluid=True, className="mt-1")

### Page 2 

In [14520]:
page_2 = dbc.Container([
    dbc.Row([
        html.H3('The most popular products', className='text-left'),
    ]),
    dbc.Row([
        dbc.Col([
            html.H4('Profit by sales'),
            dcc.Dropdown(id='category-for-product',
                         options=[{'label': label, 'value': label} for label in df['Product Name'].unique()],
                         value=['Staples', 'Staple remover'],
                         multi=True,
                         searchable=True,
                         placeholder="Select products or categories...",
                         clearable=True
                         ),
            dcc.Checklist(['discount','no discount'],
                           value = ['discount'],
                           id = 'checkboxer-for-discount',
                           ),
            dbc.Button("Submit", color="primary", className="me-1",
                       id='submitiion-button-for-graphs-page2',n_clicks=0),
            dcc.Graph(figure={}, id='profit-sale-product-compare', style={'height': '40vh'}),
            dcc.Graph(figure={}, id='product-impact-linear', style={'height': '40vh'}),
            dcc.RangeSlider(id='range-slider-impact',
                            min=0,
                            max=1410, step=30,
                            value=[0,1410],
                            marks=None)
        ], width=6),
        dbc.Col([
            html.H4('Choose category'),
            dcc.Dropdown(
                options=[{'label': category, 'value': category} for category in ['Technology', 'Furniture', 'Office Supplies']],
                value=['Technology', 'Furniture', 'Office Supplies'],
                multi=True,
                id='category-for-products',
            ),
            dcc.Graph(figure={}, id='products', style={'height': '90vh'}),  # Полная высота колонки
            dcc.Slider(2, 15, 1, value=7, id='slider-of-products'),
        ], width=6)
    ], style={'height': '100vh'}),
    dbc.Row([
        html.Hr(style={'margin-top': '40px', 'margin-bottom': '20px','border-width': '3px'})
    ])
], fluid=True, className="mt-4")


In [14521]:
app.layout = dbc.Container([
    dcc.Location(id='url', refresh=False),
    dbc.NavbarSimple(
        brand="Superstore",
        brand_style={'color': fontcolor1, 'font-size': '36px', 'font-weight': 'bold'},  # Увеличение размера текста и добавление жирности
        brand_href="/",
        color=fontcolor2,
        dark=True,
        expand='lg',  # Расширение на больших экранах
        style={'width': '100%'},  # Задание ширины навбара
        children=[
            dbc.NavItem(dcc.Link(children='Category Sales', href='/', className='nav-link', style={'color': fontcolor1, 'font-size': '18px'})),
            dbc.NavItem(dcc.Link('Sales map', href='/page-1', className='nav-link', style={'color': fontcolor1, 'font-size': '18px'})),
            dbc.NavItem(dcc.Link('Products', href='/page-2', className='nav-link', style={'color': fontcolor1, 'font-size': '18px'})),
        ]
    ),
    html.Div(id='page-content')
], fluid=True)


In [14522]:
@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/':
        return page_0
    elif pathname == '/page-1':
        return page_1
    elif pathname == '/page-2':
        return page_2

In [14523]:
@callback(
    Output('sub-categories','figure'),
    Output('one-sales-profit','figure'),
    Output('categories','figure'),
    Input('categories','clickData'),
    Input('sales_or_profit','value'),
    Input('range-slider','value'),
    Input('coef_or_profit','value')
)
def update_bar_char(clickData,val,date,coef_or_profit):
    start_date = pd.to_datetime('2014-01-01') + pd.to_timedelta(date[0], unit='D')
    end_date = pd.to_datetime('2014-01-01') + pd.to_timedelta(date[1], unit='D')
    df['Order Date'] = pd.to_datetime(df['Order Date'])
    df_subs_cat = df[df['Order Date'].between(start_date,end_date)].drop('Order Date',axis=1).\
    groupby(['Sub-Category','Category']).sum()[['Sales','Quantity','Profit']].reset_index()
    if clickData == None:
         selected_category = 'Furniture'
    else :
        selected_category = clickData['points'][0]['label']       
    filtered_df = df_subs_cat[df_subs_cat['Category'] == selected_category]    
    
    fig = px.bar(
    filtered_df.sort_values(by=val),
    y='Sub-Category',
    x=val,
    title=f'{val.capitalize()} in {selected_category}',
    color_discrete_sequence=px.colors.sequential.Viridis,  # Цветовая схема Viridis
    )

    # Настройка внешнего вида графика
    fig.update_traces(
    texttemplate='%{x:.2s}',  # Формат текста на столбцах с сокращением больших чисел (k, M)
    textposition='inside',  # Размещение текста внутри столбцов
    insidetextanchor='middle',  # Выравнивание текста по центру столбца
    marker_line_color='#FFFFFF',  # Цвет обводки столбцов
    marker_line_width=1.5,  # Толщина обводки столбцов
    showlegend=False  # Скрыть легенду, так как она дублирует категории оси Y
    )

    # Настройка заголовка, осей и фона
    fig.update_layout(
    title=dict(
        text=f'{val.capitalize()} in {selected_category}',
        font=dict(size=24, color= fontcolor2, family='Arial', weight='bold'),  # Настройка шрифта заголовка
        x=0.5,  # Центрирование заголовка
    ),
    xaxis=dict(
        title=val.capitalize(),  # Название оси X
        title_font=dict(size=20, color=fontcolor2, family='Arial'),  # Настройка шрифта для оси X
        tickfont=dict(size=12, color=fontcolor2),  # Настройка шрифта для меток
        showgrid=True,  # Показать сетку на оси X
        gridcolor='#d9e3f1',  # Цвет сетки оси X
    ),
    yaxis=dict(
        title='Sub-Category',  # Название оси Y
        title_font=dict(size=20, color=fontcolor2, family='Arial'),  # Настройка шрифта для оси Y
        tickfont=dict(size=12, color=fontcolor2),  # Настройка шрифта для меток
        showgrid=False  # Убрать сетку на оси Y
    ),
    margin=dict(t=40, b=80, l=40, r=40),  # Отступы от краев графика
    paper_bgcolor='#d9e3f1',  # Фоновый цвет всей области графика
    plot_bgcolor='#d9e3f1',   # Фоновый цвет области построения графика
    )



    filtered_df2 = filtered_df.copy()
    filtered_df2['In %'] = (filtered_df2['Profit']/filtered_df2['Sales'] +1) * 100
    filtered_df2['Average Profit'] = filtered_df2['Profit']/filtered_df2['Quantity']
    if coef_or_profit == 'In %':
        suffix = "%"
    else:
        suffix = "$"  
    if coef_or_profit == 'In %':
        texttempl2 = '%{y:.2f}%'
    else:
        texttempl2 =  '%{y:.2f}$' 
    fig2 = px.bar(
    filtered_df2.sort_values(by=coef_or_profit),
    x='Sub-Category',
    y=coef_or_profit,
    title='Profit by One Sale',
    color_discrete_sequence=px.colors.sequential.Viridis,  # Цветовая схема Viridis
    )

    # Настройка внешнего вида графика
    fig2.update_traces(
    texttemplate=texttempl2,  # Формат текста на столбцах (2 знака после запятой)
    textposition='inside',  # Размещение текста над столбцами
    marker_line_color='#FFFFFF',  # Цвет обводки столбцов
    insidetextanchor='middle',  # Выравнивание текста по центру столбца
    marker_line_width=1.5,  # Толщина обводки столбцов
    showlegend=False  # Скрыть легенду, так как она дублирует категории оси X
    )

    # Настройка заголовка, осей и фона
    fig2.update_layout(
    title=dict(
        text='Profit by One Sale per Sub-Category',
        font=dict(size=24, color= fontcolor2, family='Arial', weight='bold'),  # Настройка шрифта заголовка
        x=0.5,  # Центрирование заголовка
        pad=dict(b=100)
    ),
    xaxis=dict(
        title='Sub-Category',
        tickangle=-45,  # Поворот текста меток оси X
        title_font=dict(size=20, color=fontcolor2, family='Arial'),  # Настройка шрифта для оси X
        tickfont=dict(size=12, color=fontcolor2),  # Настройка шрифта для меток
        showgrid=False  # Убрать сетку на оси X
    ),
    yaxis=dict(
        title=coef_or_profit.replace('_', ' ').capitalize(),  # Название оси Y
        title_font=dict(size=20, color=fontcolor2, family='Arial'),  # Настройка шрифта для оси Y
        tickfont=dict(size=12, color=fontcolor2),  # Настройка шрифта для меток
        ticksuffix = suffix,
        showgrid=True,  # Показать сетку на оси Y
        gridcolor='#d9e3f1',  # Цвет сетки оси Y
    ),
    margin=dict(t=40, b=80, l=40, r=40),  # Отступы от краев графика
    paper_bgcolor='#d9e3f1',  # Фоновый цвет всей области графика
    plot_bgcolor='#d9e3f1',   # Фоновый цвет области построения графика
    )

    
    fig3 = px.pie(
    data_frame=df_subs_cat,
    values=val,
    names='Category',
    title='Category Distribution',
    hole=.35,  # Размер внутренней дырки для создания полукруговой диаграммы
    color_discrete_sequence=px.colors.sequential.Viridis,  # Выбор цветовой схемы
    )

    # Настройка внешнего вида графика
    fig3.update_traces(
    textinfo='percent+label',  # Отображение меток и процентов на диаграмме
    textposition='inside',  # Размещение текста внутри сегментов
    textfont_size=14,  # Размер шрифта текста
    marker=dict(line=dict(color='#FFFFFF', width=2)),  # Обводка сегментов диаграммы
    pull=[0.05 if i == max(df_subs_cat[val]) else 0 for i in df_subs_cat[val]]  # Вытягивание самого большого сегмента
    )

    # Настройка заголовка и легенды
    fig3.update_layout(
    title=dict(
        text='Category Distribution of Sales',
        font=dict(size=24, color= fontcolor2, family='Arial', weight='bold'),  # Настройка жирного шрифта и цвета заголовка
        x=0.5,
    ),
    showlegend = False,
    margin=dict(t=40, b=40, l=40, r=40),
    paper_bgcolor='#d9e3f1',
    plot_bgcolor='#d9e3f1',
    )
    return fig,fig2,fig3

In [14524]:
@callback(
    Output('text-card1','children'),
    Output('text-card2','children'),
    Output('text-card3','children'),
    Output('text-card4','children'),
    Output('text-card5','children'),
    Output('text-card6','children'),
    Output('text-card7','children'),
    Output('text-card8','children'),
    Input('range-slider','value')
)
def cards(date):
    start_date = pd.to_datetime('2014-01-01') + pd.to_timedelta(date[0], unit='D')
    end_date = pd.to_datetime('2014-01-01') + pd.to_timedelta(date[1], unit='D')
    df_ = df.copy()
    df_['Order Date'] = pd.to_datetime(df_['Order Date'])
    df_subs_cat = df_[df_['Order Date'].between(start_date,end_date)].drop('Order Date',axis=1).\
    groupby(['Sub-Category','Category']).sum()[['Sales','Quantity','Profit']].reset_index()
    sales = df_subs_cat['Sales'].sum().round()
    profit = df_subs_cat['Profit'].sum().round()
    df_ = df.copy()
    df_['Order Date'] = pd.to_datetime(df_['Order Date'])
    df_ = df_[df_['Order Date'].between(start_date,end_date)]
    aov = (df_['Sales'].sum() / df_['Order ID'].nunique()).round(2)
    top_customer = df_.groupby('Customer Name')['Sales'].sum().idxmax()
    top_customer_sales = df_.groupby('Customer Name')['Sales'].sum().max().round()
    repeat_customers = df_['Customer ID'].value_counts().loc[lambda x: x > 1].count()
    total_customers = df_['Customer ID'].nunique()
    repeat_customer_rate = (repeat_customers / total_customers) * 100
    df_['Order Date'] = pd.to_datetime(df_['Order Date'])
    df_['Ship Date'] = pd.to_datetime(df_['Ship Date'])
    df_['Shipping Time'] = (df_['Ship Date'] - df_['Order Date']).dt.days
    average_shipping_time = df_['Shipping Time'].mean().round(2)
    df_ = df_.groupby('Order Date')[['Sales','Profit']].sum().reset_index()
    day_sales = df_['Sales'].mean().round()
    day_profit = df_['Profit'].mean().round()
    return f'{sales} $',f'{profit} $',f'{day_profit} $',f'{day_sales} $',\
    aov,f'{top_customer}-{top_customer_sales}$',f'{repeat_customer_rate:.2f}%',\
    f'{average_shipping_time} days'


In [14525]:
@callback(
    Output('slider-output','children'),
    Input('range-slider','value'),
)
def range_slider(val):
    start_date = pd.to_datetime('2014-01-01') + pd.to_timedelta(val[0], unit='D')
    end_date = pd.to_datetime('2014-01-01') + pd.to_timedelta(val[1], unit='D')
    return f'Selected range: {start_date.date()} to {end_date.date()}'

In [14526]:
@callback(
Output('map','figure'),
Input('range-slider2','value'),
)
def mapp_graph(val_):
    df_ = df_data.drop('Order Date',axis=1).\
    groupby(['State','date_num']).sum().reset_index()
    df_['date_num'] = df_['date_num'].apply(lambda x: round((x/200))*200)
    df_['State code'] = df_['State'].map(state_codes)
    df_ = df_.groupby(['State','State code','date_num']).sum().reset_index()
    df_ = df_[df_['date_num'].between(val_[0],val_[1])]
    df_ = df_.drop('date_num',axis=1).groupby(['State','State code']).sum().reset_index()
    mapp = px.choropleth(
        df_,
        locations='State code',
        locationmode="USA-states",
        color='Sales',
        scope="usa",
        hover_name='State',
        hover_data={'Sales': True, 'Profit': True},
        color_continuous_scale='Viridis'
)
    return mapp

In [14527]:
@app.callback(
    Output('map_state', 'figure'),
    Input('map', 'clickData')
)
def update_bar_chart(clickData):
    if clickData is None:
        state_name = 'California'
    else:
        state_name = clickData['points'][0]['hovertext']
    filtered_df = df_states_w_city[df_states_w_city['State'] == state_name].sort_values(by='Sales')
    fig = px.bar(filtered_df, x='City', y='Sales')
    fig.update_yaxes(range=[0, 10000])
    return fig

In [14528]:
@callback(
    Output('sub-category-in-state','figure'),
    Input('map', 'clickData'),
)
def update_bar_subcategory(clickData):
    if clickData is None:
        state_name = 'California'
    else:
        state_name = clickData['points'][0]['hovertext']
    df_sub_category_state = df[df['State'] == state_name].drop('Order Date',axis=1).\
    groupby(by='Sub-Category').sum().reset_index()[['Sub-Category','Sales']].sort_values(by='Sales')
    fig = px.bar(df_sub_category_state, x='Sub-Category', y='Sales')
    return fig


In [14529]:
@callback(
Output('products','figure'),
Input('slider-of-products','value'),
Input('category-for-products','value')
)
def product_slider(quantity,category):
    df_ = df[df['Category'].isin(category)]
    df_ = df_[['Product Name','short name']].value_counts()[:quantity].reset_index()
    fig = px.pie(df_,values='count',names='short name')
    return fig

In [14530]:
@callback(
Output('profit-sale-product-compare','figure'),
Input('submitiion-button-for-graphs-page2','n_clicks'),
State('checkboxer-for-discount','value'),
State('category-for-product','value'),
)
def profit_sale_product_compare(_,discount,selected_product):
     df_ = df[df['Product Name'].isin(selected_product)]
     df_['Discount_presence'] = df_['Discount'].apply(lambda x: 'discount' if x>0 else 'no discount')
     df_ = df_[['Sales','Profit','Product Name','short name','Discount_presence']].groupby(['Product Name','short name','Discount_presence']).sum().reset_index()
     df_['Impact'] = df_['Profit']/df_['Sales']
     df_['Impact'] = df_['Impact'] + 1
     df_ = df_[df_['Discount_presence'].isin(discount)]
     fig = px.bar(df_,x='short name',y='Impact')
     return fig


In [14531]:
@callback(
    Output('product-impact-linear','figure'),
    Input('submitiion-button-for-graphs-page2','n_clicks'),
    State('range-slider-impact','value'),
    State('category-for-product','value'),
    State('checkboxer-for-discount','value'),
)
def range_slider_impact(_,val_,selected_product,discount):
    df_ = df.copy()
    df_['Order Date'] = pd.to_datetime(df_['Order Date'])
    df_ = df_[['Order Date','Sales','Profit','Product Name','short name','Discount']]
    df_['Discount_presence'] = df_['Discount'].apply(lambda x: 'discount' if x>0 else 'no discount')
    df_ = df_.groupby(['Order Date','Product Name','short name','Discount_presence'])[['Sales','Profit']].sum().reset_index()
    df_['date_num'] = (df_['Order Date']-pd.to_datetime('2014-01-03')).dt.days
    df_ = df_.sort_values(by='Order Date')
    df_['Impact'] = (df_['Profit']/df_['Sales']) + 1
    df_['Impact'] = df_['Impact'].round(2)
    df_ = df_[df_['Product Name'].isin(selected_product)]
    df_ = df_[df_['date_num'].between(val_[0],val_[1])]
    df_ = df_[df_['Discount_presence'].isin(discount)]
    fig = px.line(df_,y='Impact',x='date_num',color='short name')
    return fig

In [14532]:
if __name__ == "__main__":
    app.run_server(debug=True,port=8051)

C:\Users\Acer\AppData\Local\Temp\ipykernel_61976\1886480711.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Acer\AppData\Local\Temp\ipykernel_61976\1886480711.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Acer\AppData\Local\Temp\ipykernel_61976\1886480711.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d